# SEQUENCE-TO-SEQUENCE NMT WITH ATTENTION MECHANISM

# DATA

In [1]:
import numpy as np
import re
import string
import unicodedata
from unicodedata import normalize
import sys
import indicnlp
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import array
from numpy import asarray
from numpy import zeros
# from keras.models import Model
# from keras.layers import Input, LSTM, GRU, Dense, Embedding
from tensorflow.keras.utils import plot_model
import numpy as np
# !pip install tensorflow-addons==0.11.2
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
import gensim
from gensim.models import Word2Vec
from tensorflow import keras
# !pip install indic-nlp-library : this is necessary, without this common wont be found and correct indic nlp wont get installed
import os
import io
import time
from gensim.models.fasttext import FastText
INDIC_NLP_LIB_HOME=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\GLOve_large_dataset\anoopkunchukuttan-indic_nlp_library-eccde81"
INDIC_NLP_RESOURCES=r"C:\Users\sudha\Desktop\ALL_PROJECTS\NMT_PROJECTS\Language_Translation_Chat_Bot\LSTM_MODELS\GLOve_large_dataset\indic_nlp_resources-master"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

# LOAD DATA

In [2]:
def load_doc(doc):
    file = open(doc , mode = 'rt' , encoding = 'utf -8')
    text = file.read()
    file.close()
    return text

def clean_english_data(lines):
    re_print = re.compile('[^%s]' %re.escape(string.printable))
    table = str.maketrans('' , '' , string.punctuation)
    
    
    clean_eng_lines = []
    ans = list()
    ans.append(lines)
    
    for line in ans:
        line = normalize('NFD' , line).encode('ascii' , 'ignore')
        line = line.decode('UTF-8')
        line = line.split()
        line = [word.lower() for word in line]
        line = [word.translate(table) for word in line]
        line = [word for word in line if word.isalpha()]
        line = ' '.join(line)
        line = '<start> ' + line +' <end>'
        clean_eng_lines.append(line)
    return clean_eng_lines

def clean_text(line):
    text = line
    text = text.replace(u',' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u'(' ,'')
    text = text.replace(u')' ,'')
    text = text.replace(u'"' ,'')
    text = text.replace(u':' ,'')
    text = text.replace(u"'" ,'')
    text = text.replace(u"'" ,'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text = re.sub('[a-zA-Z]' , '' , text)
    text = re.sub('[0-9+\-*/.%&!]' , '' , text)
    return text

def clean_hindi_data(lines):
    clean_hin_lines = []
    ans = list()
    ans.append(lines)
    for line in ans:
        remove_nuktas = True
        factory = IndicNormalizerFactory()
        normalizer = factory.get_normalizer("hi" , remove_nuktas = True )
        line = clean_text(line)
        tokens = list()
        for t in indic_tokenize.trivial_tokenize(line):
            tokens.append(t)
        line = tokens
        line = [ word.lower() for word in line]
        line = [word for word in line if not re.search(r'\d', word)]
        line = ' '.join(line)
        line = '<start> ' + line +' <end>'
        clean_hin_lines.append(line)
        
    return clean_hin_lines   


def to_pairs(text):
    lines = text.strip().split('\n')
    
    eng_hin_pairs = [line.split('\t') for line in lines]
    eng_hin_pairs = np.delete(eng_hin_pairs, [0,1,2,3,4] , axis = 1)
    eng_hin_pairs = eng_hin_pairs.tolist()
    clean_data = list()
    for pair in eng_hin_pairs:
        temp_data = list()
        for i, sentence in enumerate(pair):
            if i == 0:
                x = clean_english_data(sentence)
                temp_data.append(x)
            if i == 1:
                y = clean_hindi_data(sentence)
                temp_data.append(y)
        clean_data.append(temp_data)
    return eng_hin_pairs, clean_data

doc = load_doc("hindi-visual-genome-train.txt")
pairs , clean_data = to_pairs(doc)

In [3]:
clean_data

[[['<start> it is an indoor scene <end>'],
  ['<start> यह एक इनडोर दृश्य है <end>']],
 [['<start> computer screens turned on <end>'],
  ['<start> कंप्यूटर स्क्रीन चालू <end>']],
 [['<start> man has short hair <end>'],
  ['<start> आदमी के छोटे बाल हैं <end>']],
 [['<start> photo album open on an adults lap <end>'],
  ['<start> एक वयस्क की गोद में फोटो एल्बम खुला <end>']],
 [['<start> there is a group of girls beside the black car <end>'],
  ['<start> काली कार के पास लड़कियों का एक समूह है <end>']],
 [['<start> child in a stroller <end>'],
  ['<start> एक घुमक्कड़ में बच्चा <end>']],
 [['<start> tall metal lightpost <end>'],
  ['<start> लंबा धातु प्रकाश पोस्ट <end>']],
 [['<start> wall is painted white <end>'],
  ['<start> दीवार सफेद रंग की है <end>']],
 [['<start> there are several pictures on the wall <end>'],
  ['<start> दीवार पर कई चित्र हैं <end>']],
 [['<start> woman facing the ocean <end>'],
  ['<start> सागर का सामना कर रही महिला <end>']],
 [['<start> this is an office layout <end>

In [4]:
# shuffling values
import random
BUFFER_SIZE = 28930
BATCH_SIZE = 64
num_examples = 20000
# train_dataset = clean_data[:n_train]
# val_dataset = clean_data[n_train:]
# # Let's limit the #training examples for faster training

# train_dataset = random.shuffle(train_dataset)
# val_dataset = random.shuffle(val_dataset)
# type(clean_data[0][1])

In [5]:
# train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [6]:
clean_hindi_lines = []
clean_english_lines = []
for i in range(len(clean_data)):
    clean_hindi_lines.append(clean_data[i][1])
    clean_english_lines.append(clean_data[i][0])
    


In [7]:
clean_hindi = list()
for i in range(len(clean_hindi_lines)):
    clean_hindi.append(clean_hindi_lines[i][0])
    
clean_english = list()
for i in range(len(clean_english_lines)):
    clean_english.append(clean_english_lines[i][0])


In [8]:
clean_english_lines

[['<start> it is an indoor scene <end>'],
 ['<start> computer screens turned on <end>'],
 ['<start> man has short hair <end>'],
 ['<start> photo album open on an adults lap <end>'],
 ['<start> there is a group of girls beside the black car <end>'],
 ['<start> child in a stroller <end>'],
 ['<start> tall metal lightpost <end>'],
 ['<start> wall is painted white <end>'],
 ['<start> there are several pictures on the wall <end>'],
 ['<start> woman facing the ocean <end>'],
 ['<start> this is an office layout <end>'],
 ['<start> four metallic chairs <end>'],
 ['<start> clutter is on a table <end>'],
 ['<start> a white microwave oven <end>'],
 ['<start> white suv driving through intersection <end>'],
 ['<start> person crossing street with umbrella <end>'],
 ['<start> man in gray pants leaning on building <end>'],
 ['<start> window on the building <end>'],
 ['<start> a man standing in between cars <end>'],
 ['<start> painting hanging on wall <end>'],
 ['<start> leg of the chair <end>'],
 ['<s

In [9]:
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(padded_tensor_hin, padded_tensor_eng, test_size=0.2, random_state = 42)
input_train , input_val, target_train, target_val = train_test_split(clean_english , clean_hindi, test_size = 0.2, random_state = 42)

In [10]:
hin_lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
hin_lang_tokenizer.fit_on_texts(clean_hindi)
hin_dict = hin_lang_tokenizer.word_index
tensor_hin = hin_lang_tokenizer.texts_to_sequences(target_train) 
padded_tensor_hin = tf.keras.preprocessing.sequence.pad_sequences(tensor_hin, padding='post')
# print(tensor)
# len(hin_dict)
target_tensor_train =  padded_tensor_hin
print(len(padded_tensor_hin))
print(len(hin_dict))

23144
7013


In [11]:
eng_lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>')
eng_lang_tokenizer.fit_on_texts(clean_english)
eng_dict = eng_lang_tokenizer.word_index
tensor_eng = eng_lang_tokenizer.texts_to_sequences(input_train) 
padded_tensor_eng = tf.keras.preprocessing.sequence.pad_sequences(tensor_eng, padding='post')
input_tensor_train = padded_tensor_eng
# print(tensor)
# len(hin_dict)
print(len(padded_tensor_eng))
print(len(eng_dict))

23144
5553


In [12]:
# train test split
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(padded_tensor_eng, padded_tensor_hin, test_size=0.2, random_state = 42)


In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
# val_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val))
# val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [14]:
train_dataset

<BatchDataset shapes: ((64, 26), (64, 34)), types: (tf.int32, tf.int32)>

In [15]:
len(target_tensor_train[0])

34

In [16]:
example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 26]), TensorShape([64, 34]))

In [17]:
# some important parameters
vocab_inp_size = len(eng_lang_tokenizer.word_index)+1
vocab_tar_size = len(hin_lang_tokenizer.word_index)+1
max_length_input = len(input_tensor_train[0])
max_length_output = len(target_tensor_train[0])

embedding_dim = 100
units = 1024
steps_per_epoch = num_examples//BATCH_SIZE

In [18]:
print("max_length_english, max_length_hindi, vocab_size_english, vocab_size_hindi")
max_length_input, max_length_output, vocab_inp_size, vocab_tar_size

max_length_english, max_length_hindi, vocab_size_english, vocab_size_hindi


(26, 34, 5554, 7014)

In [19]:
# hindi embedding word2vec
# Load pre-trained Word2Vec model.
model = gensim.models.Word2Vec.load('fasttext\hi-d100-m2-fasttext.model')

In [20]:
embeddings_dictionary_h = {}
for word in hin_lang_tokenizer.word_index:
    try:
        embeddings_dictionary_h[word] = model.wv.get_vector(word)
    except:
        pass
    
print("the no of key-value pairs:"  , len(embeddings_dictionary_h))
embeddings_dictionary_h

the no of key-value pairs: 7013


{'<OOV>': array([-0.13615343, -0.0276705 , -0.32373258, -0.33983436, -0.25074765,
         0.1136819 ,  0.06906117,  0.13111897,  0.37115008, -0.18939647,
         0.33253947, -0.09550375,  0.03981497, -0.00756057,  0.0919835 ,
         0.10029203,  0.00405759,  0.0080026 ,  0.01395009,  0.03374427,
         0.11002628, -0.32754344, -0.29076695, -0.15364802, -0.10591869,
         0.0816481 , -0.05164264,  0.18296564, -0.03330514, -0.3384538 ,
        -0.0645583 ,  0.3143947 , -0.09430255,  0.00978532,  0.1626066 ,
         0.20863284, -0.25920966, -0.24931912, -0.08786643,  0.08610689,
         0.13213517,  0.08543954,  0.18012294,  0.05626457,  0.23760124,
         0.38632652, -0.05079996,  0.03733306,  0.05593262, -0.08317049,
         0.07317774,  0.03252045,  0.22009286, -0.24432637,  0.01289156,
         0.09061732,  0.1443138 , -0.12644455,  0.22806767,  0.24784693,
         0.1422608 , -0.05146387, -0.04265721,  0.00522736,  0.2764139 ,
        -0.17195396,  0.15091114, -0.05463

In [21]:
num_words = vocab_tar_size
embedding_matrix_h = zeros((vocab_tar_size, 100))
for word, index in hin_lang_tokenizer.word_index.items(): #input_lang_word2idx=1847
    embedding_vector = embeddings_dictionary_h.get(word)
    if embedding_vector is not None:
        embedding_matrix_h[index] = embedding_vector

In [22]:
len(embedding_matrix_h)

7014

In [23]:
# preparing english lines for w2v
eng = []
for line in clean_english_lines:
    for l in line:
        l = l.split(" ")
    eng.append(l)

eng

[['<start>', 'it', 'is', 'an', 'indoor', 'scene', '<end>'],
 ['<start>', 'computer', 'screens', 'turned', 'on', '<end>'],
 ['<start>', 'man', 'has', 'short', 'hair', '<end>'],
 ['<start>', 'photo', 'album', 'open', 'on', 'an', 'adults', 'lap', '<end>'],
 ['<start>',
  'there',
  'is',
  'a',
  'group',
  'of',
  'girls',
  'beside',
  'the',
  'black',
  'car',
  '<end>'],
 ['<start>', 'child', 'in', 'a', 'stroller', '<end>'],
 ['<start>', 'tall', 'metal', 'lightpost', '<end>'],
 ['<start>', 'wall', 'is', 'painted', 'white', '<end>'],
 ['<start>',
  'there',
  'are',
  'several',
  'pictures',
  'on',
  'the',
  'wall',
  '<end>'],
 ['<start>', 'woman', 'facing', 'the', 'ocean', '<end>'],
 ['<start>', 'this', 'is', 'an', 'office', 'layout', '<end>'],
 ['<start>', 'four', 'metallic', 'chairs', '<end>'],
 ['<start>', 'clutter', 'is', 'on', 'a', 'table', '<end>'],
 ['<start>', 'a', 'white', 'microwave', 'oven', '<end>'],
 ['<start>', 'white', 'suv', 'driving', 'through', 'intersection', '

In [24]:
### english word embeddings
# Defining values for parameters
embedding_size = 100
window_size = 5
min_word = 5
down_sampling = 1e-2

model_e = FastText(eng,vector_size = embedding_size,
                       window = window_size,
                       min_count = min_word,
                       sample = down_sampling,
                       workers = 4,
                       sg = 1,
                       epochs =100)

In [25]:
model_e.save("FT_model")
#load saved gensim fastext model
model_e = Word2Vec.load("FT_model")

In [26]:
embeddings_dictionary = {}
for word in eng_lang_tokenizer.word_index:
    try:
        embeddings_dictionary[word] = model_e.wv.get_vector(word)
    except:
        pass
    
print("the no of key-value pairs:"  , len(embeddings_dictionary))
embeddings_dictionary

the no of key-value pairs: 5553


{'<OOV>': array([ 8.68141593e-04,  1.76327696e-04,  1.37837883e-03,  4.35418217e-04,
        -3.63037718e-04, -2.33888757e-04, -2.09328299e-03, -7.14867274e-05,
         3.82969482e-03,  3.13256495e-03,  2.08807527e-03, -2.63648049e-04,
        -2.72413861e-04, -3.60388978e-04,  3.60667560e-04,  1.65625359e-03,
         1.00784691e-03,  9.98774660e-04,  4.21727163e-04,  2.72886944e-03,
        -2.27275174e-04, -1.05543586e-03,  6.88576023e-04, -4.71114618e-04,
         3.02961154e-04,  1.29212148e-03, -1.38463895e-03,  9.70724854e-04,
        -2.01766510e-04, -1.55456655e-03, -2.93521705e-04, -1.26592291e-03,
        -1.47013518e-03,  1.13491423e-03, -1.04494102e-03,  7.48905994e-04,
         8.93076503e-05, -1.23496354e-03, -2.82389903e-03, -1.95336388e-03,
        -1.37665687e-04, -1.53816433e-03,  9.05005902e-04,  7.73039530e-04,
         4.25788137e-04,  7.31268956e-04,  1.24309107e-03,  7.39316165e-04,
        -3.49453709e-04, -1.03528684e-04,  2.06189253e-03, -1.27490121e-03,
   

In [27]:
num_words = vocab_inp_size
embedding_matrix = zeros((vocab_inp_size, 100))
for word, index in eng_lang_tokenizer.word_index.items(): #input_lang_word2idx=1847
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [28]:
class Encoder(tf.keras.Model):
    
    
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz, embedding_matrix):
        
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        embedding_matrix = embedding_matrix
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights = [embedding_matrix])

        ##-------- LSTM layer in Encoder ------- ##
        self.lstm_layer = tf.keras.layers.LSTM(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')



    def call(self, x, hidden):
        
        x = self.embedding(x)
        output, h, c = self.lstm_layer(x, initial_state = hidden)
        return output, h, c

    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]

In [29]:
## Test Encoder Stack

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, embedding_matrix)


# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_h, sample_c = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h.shape))
print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c.shape))

Encoder output shape: (batch size, sequence length, units) (64, 26, 1024)
Encoder h vecotr shape: (batch size, units) (64, 1024)
Encoder c vector shape: (batch size, units) (64, 1024)


In [30]:
class Decoder(tf.keras.Model):
    def __init__(self, embedding_matrix_h, vocab_size_h, embedding_dim, dec_units, batch_sz, attention_type = 'luong'):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.attention_type = attention_type
        embedding_matrix_h = embedding_matrix_h
        
        #Embedding layer
        
        self.embedding = tf.keras.layers.Embedding(vocab_size_h, embedding_dim, weights = [embedding_matrix_h])
        
        #final dense layer on which softmax will be applied
        self.fc = tf.keras.layers.Dense(vocab_size_h)
        
        # define the fundamental cell for decoder recurrent structure
        
        self.decoder_rnn_cell = tf.keras.layers.LSTMCell(self.dec_units)
        
        # sampler 
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        
        # create attention mechanism with memory = None
        self.attention_mechanism = self.build_attention_mechanism(self.dec_units, None, self.batch_sz*[max_length_input], self.attention_type)
        
        # wrap attention mechanism with the fundamental rnn cell
        self.rnn_cell = self.build_rnn_cell(batch_sz)
        
        # define the decoder with respect to fundamental rnn cell
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)

        
        
    def build_rnn_cell(self, batch_sz):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell, 
                                      self.attention_mechanism, attention_layer_size=self.dec_units)
        return rnn_cell
    
    def build_attention_mechanism(self, dec_units, memory,memory_sequence_length, attention_type = 'luong'):
         # ------------- #
        # typ: Which sort of attention (Bahdanau, Luong)
        # dec_units: final dimension of attention outputs 
        # memory: encoder hidden states of shape (batch_size, max_length_input, enc_units)
        # memory_sequence_length: 1d array of shape (batch_size) with every element set to max_length_input (for masking purpose)
        if(attention_type == 'bahdanau'):
            return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
        
        else:
             return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
            
    def build_initial_state(self, batch_sz, encoder_state, Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
        return decoder_initial_state


    def call(self, inputs, initial_state):
        x = self.embedding(inputs)
        outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[max_length_output-1])
        return outputs        

        
    

In [31]:
# Test decoder stack

decoder = Decoder(embedding_matrix_h, vocab_tar_size, embedding_dim, units, BATCH_SIZE, 'luong')
sample_x = tf.random.uniform((BATCH_SIZE, max_length_output))
decoder.attention_mechanism.setup_memory(sample_output)
initial_state = decoder.build_initial_state(BATCH_SIZE, [sample_h, sample_c], tf.float32)


sample_decoder_outputs = decoder(sample_x, initial_state)

print("Decoder Outputs Shape: ", sample_decoder_outputs.rnn_output.shape)

Decoder Outputs Shape:  (64, 33, 7014)


In [32]:
# defing optimizer and loss function

optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
    # real shape = (BATCH_SIZE, max_length_output)
    # pred shape = (BATCH_SIZE, max_length_output, tar_vocab_size )
    cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    loss = cross_entropy(y_true=real, y_pred=pred)
    mask = tf.logical_not(tf.math.equal(real,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)  
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss

In [33]:

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [34]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    
    with tf.GradientTape() as tape:
        enc_output, enc_h, enc_c = encoder(inp, enc_hidden)


        dec_input = targ[ : , :-1 ] # Ignore <end> token
        real = targ[ : , 1: ]         # ignore <start> token

        # Set the AttentionMechanism object with encoder_outputs
        decoder.attention_mechanism.setup_memory(enc_output)

        # Create AttentionWrapperState as initial_state for decoder
        decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
        pred = decoder(dec_input, decoder_initial_state)
        logits = pred.rnn_output
        loss = loss_function(real, logits)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss

In [ ]:
# EPOCHS = 10

# for epoch in range(EPOCHS):
#     start = time.time()
#     enc_hidden = encoder.initialize_hidden_state()
#     total_loss = 0
#     # print(enc_hidden[0].shape, enc_hidden[1].shape)
    
#     for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
#         batch_loss = train_step(inp, targ, enc_hidden)
#         total_loss += batch_loss
#         if batch % 100 == 0:
#             print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
#                                                    batch,
#                                                    batch_loss.numpy()))
#    # saving (checkpoint) the model every 2 epochs
#     if (epoch + 1) % 2 == 0:
#         checkpoint.save(file_prefix = checkpoint_prefix)

#     print('Epoch {} Loss {:.4f}'.format(epoch + 1,
#                                       total_loss / steps_per_epoch))
#     print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [35]:
def evaluate_sentence(sentence):
    sentence = clean_english_data(sentence)
    string = " ".join(sentence)
    inputs = [eng_lang_tokenizer.word_index[i] for i in string.split(" ")]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_input,
                                                          padding='post')
    inputs = tf.convert_to_tensor(inputs)
    inference_batch_size = inputs.shape[0]
    result = ''
    enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
    enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)
    dec_h = enc_h
    dec_c = enc_c
    start_tokens = tf.fill([inference_batch_size], hin_lang_tokenizer.word_index['<start>'])
    end_token = hin_lang_tokenizer.word_index['<end>']
    greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()
    # Instantiate BasicDecoder object
    decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc)
    # Setup Memory in decoder stack
    decoder.attention_mechanism.setup_memory(enc_out)

    # set decoder_initial_state
    decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)


    ### Since the BasicDecoder wraps around Decoder's rnn cell only, you have to ensure that the inputs to BasicDecoder 
    ### decoding step is output of embedding layer. tfa.seq2seq.GreedyEmbeddingSampler() takes care of this. 
    ### You only need to get the weights of embedding layer, which can be done by decoder.embedding.variables[0] and pass this callabble to BasicDecoder's call() function
    decoder_embedding_matrix = decoder.embedding.variables[0]
    outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token= end_token, initial_state=decoder_initial_state)
    return outputs.sample_id.numpy()

def translate(sentence):
    result = evaluate_sentence(sentence)
    print(result)
    result = hin_lang_tokenizer.sequences_to_texts(result)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    return result, sentence

In [36]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [37]:
# removing <start> and <end> from input_val
input_val_rem = []
for i in range(len(input_val)):
    l = input_val[i].split(" ")
    l.pop(0)
    n = len(l)-1
    l.pop(n)
    s = " ".join(l)
    input_val_rem.append(s)

In [38]:
input_val_rem.index('')

3575

In [39]:
print(len(input_val_rem), len(target_val))

5786 5786


In [40]:
input_val_rem.pop(3575) , target_val.pop(3575)

('', '<start> समुद्र तट पर बाइक <end>')

In [41]:
print(len(input_val_rem), len(target_val))

5785 5785


In [42]:
input_val_rem.pop(1346)
target_val.pop(1346)

'<start>  <end>'

In [43]:
input_val_rem.pop(1655)
target_val.pop(1655)

'<start>  <end>'

In [44]:
print(len(input_val_rem) , len(target_val))

5783 5783


In [45]:
input_val_rem.pop(258)
target_val.pop(258)
print(len(input_val_rem) , len(target_val))

5782 5782


In [46]:
translation = []
for i in range(0,100):
    translated_sent = translate(input_val_rem[i])
    translation.append(translated_sent)

[[16 32  6  3]]
Input: this is the sky
Predicted translation: ['यह आकाश है <end>']
[[238  13  10 231   7  15  70   3]]
Input: boy with purple and white shoes
Predicted translation: ['बैंगनी और सफेद जूते के साथ लड़का <end>']
[[ 12  14  79  31 565   6   3]]
Input: man wearing dapper red hat
Predicted translation: ['आदमी लाल टोपी पहने रखा है <end>']
[[  54   19   11 1163    3]]
Input: green colored leaves
Predicted translation: ['हरे रंग की पत्तियों <end>']
[[1332  648   24 1482  175   44    4 1650    3]]
Input: a runner leading off of first base
Predicted translation: ['पहले आधार से होकर करने वाला एक मनोरंजन <end>']
[[ 4 14 28  3]]
Input: a red train
Predicted translation: ['एक लाल ट्रेन <end>']
[[321  14  50   5   3]]
Input: windown on dark red building
Predicted translation: ['गहरे लाल इमारत पर <end>']
[[ 12   5 183 322 322   3]]
Input: black suit jacket on man
Predicted translation: ['आदमी पर ब्लैक सूट सूट <end>']
[[ 85 341  48   3]]
Input: big grey indian elephant
Predicted translati

[[  4 177  11 166   3]]
Input: a table scene
Predicted translation: ['एक भोजन की दृश्य <end>']
[[187 295 962   3]]
Input: man carrying a surfboard
Predicted translation: ['सर्फ़बोर्ड ले जाना <end>']
[[351   7  82  65   3]]
Input: snow on top of mountain
Predicted translation: ['पहाड़ के ऊपर बर्फ <end>']
[[1557    8   94    3]]
Input: birds in the shot
Predicted translation: ['शॉट में पक्षी <end>']
[[16  4 12  6  3]]
Input: this is a man
Predicted translation: ['यह एक आदमी है <end>']
[[ 48 160  67   6   3]]
Input: elephant standing in clearing
Predicted translation: ['हाथी ने खड़ा है <end>']
[[  39    5 5067    3]]
Input: ud on the clock
Predicted translation: ['घड़ी पर ब्रीस्टोल <end>']
[[108   9   4 147 204  53  20   3]]
Input: woman eating a slice of pizza
Predicted translation: ['पिज्जा का एक टुकड़ा खा रही महिला <end>']
[[  4 311   8 150  67   3]]
Input: serviceman standing in a line
Predicted translation: ['एक पंक्ति में खड़े खड़ा <end>']
[[ 4 17  9 40  3]]
Input: head of a person


In [47]:
target_val_1 = target_val[:100]
for i in range(len(target_val_1)):
    target_val_1[i] = target_val_1[i].split(" ")[1:-1]
    
t = []
for i in range(len(translation)):
    t.append(translation[i][0])    

t1=[]
for i in range(len(t)):
    l = t[i][0].split(" ")[:-1]
    s = " ".join(l)
    t1.append(s)

In [48]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
avg_1g_score = 0
for i in range(len(t1)):
    candidate = t1[i].split(" ")
    reference = [target_val_1[i]]
    score = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
#     print(score)
    avg_1g_score = avg_1g_score + score

avg_1g_score = avg_1g_score/len(t1)
avg_1g_score

C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\sudha\anaconda3\envs\rtx_3060\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram

0.6897323008080463

In [49]:
avg_2g_score = 0
for i in range(len(t1)):
    candidate = t1[i].split(" ")
    reference = [target_val_1[i]]
    score = sentence_bleu(reference, candidate, weights=(0, 1, 0, 0))
#     print(score)
    avg_2g_score = avg_2g_score + score

avg_2g_score = avg_2g_score/len(t1)
avg_2g_score 

0.5162560874365772

In [50]:
avg_ng_score = 0
for i in range(len(t1)):
    candidate = t1[i].split(" ")
    reference = [target_val_1[i]]
    score = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
#     print(score)
    avg_ng_score = avg_ng_score + score

avg_ng_score = avg_ng_score/len(t1)
avg_ng_score

0.24544146682250859